# PFPE repeatability

In [10]:
import pandas as pd
import numpy as np
import pingouin as pg
import os

def concordance_correlation_coefficient(y_true, y_pred):
    """Compute the Concordance Correlation Coefficient (CCC) between two arrays."""
    cor = np.corrcoef(y_true, y_pred)[0, 1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred) ** 2
    return numerator / denominator

def compute_ccc_for_features(df1, df2, feature_cols):
    ccc_results = pd.DataFrame({
        'Feature': feature_cols,
        'CCC': [concordance_correlation_coefficient(df1[col], df2[col]) for col in feature_cols]
    })
    return ccc_results


df_raw = pd.read_excel(r"PATH/TO/EXCEL.xlsx").iloc[:, 1:]
df_raw['TR_label'] = df_raw['Scan_Name'].str[6:]
df_raw['exp_type'] = df_raw['Image'].apply(lambda x: os.path.normpath(x).split(os.path.sep)[4])

rad_start = df_raw.columns.get_loc('original_shape_Elongation')
rad_end = df_raw.columns.get_loc('lbp-3D-k_ngtdm_Contrast') + 1
feature_cols = df_raw.columns[rad_start:rad_end]

# Intrasession
def process_test_retest(df):
    test_df = df[df['TR_label'] == 'Test'].copy()
    retest_df = df[df['TR_label'] == 'Retest'].copy()
    test_df['Unique_id'] = test_df['Scan_Name'].str[:6] + test_df['Segment Name']
    retest_df['Unique_id'] = retest_df['Scan_Name'].str[:6] + retest_df['Segment Name']
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, feature_cols)

ccc_tr = process_test_retest(df_raw[df_raw['exp_type'] != 'Parameters'])

# Time Repeatability
def process_time(df):
    df['Unique_id'] = df['Scan_Name'] + '_' + df['Segment Name']
    test_df = df[~df['Scan_Name'].str.contains('Retest')]
    retest_df = df[df['Scan_Name'].str.contains('Retest')]
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, feature_cols)

ccc_time = process_time(df_raw[df_raw['exp_type'] == 'R_Time'].copy())


# In Vivo Test-Retest
def process_in_vivo(filepath):
    df = pd.read_csv(filepath).iloc[:, 1:]
    df['TR_type'] = df['Scan_Name'].str[3:]
    test_df = df[df['TR_type'] == 'Test'].copy()
    retest_df = df[df['TR_type'] == 'Retest'].copy()
    test_df['Unique_id'] = test_df['Scan_Name'].str[:6] + '_' + test_df['Segment Name']
    retest_df['Unique_id'] = retest_df['Scan_Name'].str[:6] + '_' + retest_df['Segment Name']
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, df.columns[rad_start:rad_end])

ccc_tr.to_excel('PFPE_intrasession_CCC.xlsx', index=False)
ccc_time.to_excel('PFPE_intersession_CCC.xlsx', index=False)

ccc_in_vivo = process_in_vivo('in_vivo_radiomics_features.csv')
ccc_in_vivo.to_excel('in_vivo_CCC.xlsx', index=False)

# PFCE repeatability

In [11]:
import pandas as pd
import numpy as np
import pingouin as pg
import os

def concordance_correlation_coefficient(y_true, y_pred):
    """Compute the Concordance Correlation Coefficient (CCC) between two arrays."""
    cor = np.corrcoef(y_true, y_pred)[0, 1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred) ** 2
    return numerator / denominator

def compute_ccc_for_features(df1, df2, feature_cols):
    ccc_results = pd.DataFrame({
        'Feature': feature_cols,
        'CCC': [concordance_correlation_coefficient(df1[col], df2[col]) for col in feature_cols]
    })
    return ccc_results


df_raw = pd.read_excel(r"PATH/TO/EXCEL.xlsx").iloc[:, 1:]
df_raw['TR_label'] = df_raw['Scan_Name'].str[6:]
df_raw['exp_type'] = df_raw['Image'].apply(lambda x: os.path.normpath(x).split(os.path.sep)[4])

rad_start = df_raw.columns.get_loc('original_shape_Elongation')
rad_end = df_raw.columns.get_loc('lbp-3D-k_ngtdm_Contrast') + 1
feature_cols = df_raw.columns[rad_start:rad_end]

# Intrasession
def process_test_retest(df):
    test_df = df[df['TR_label'] == 'Test'].copy()
    retest_df = df[df['TR_label'] == 'Retest'].copy()
    test_df['Unique_id'] = test_df['Scan_Name'].str[:6] + test_df['Segment Name']
    retest_df['Unique_id'] = retest_df['Scan_Name'].str[:6] + retest_df['Segment Name']
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, feature_cols)

ccc_tr = process_test_retest(df_raw[df_raw['exp_type'] != 'Parameters'])

# Time Repeatability
def process_time(df):
    df['Unique_id'] = df['Scan_Name'] + '_' + df['Segment Name']
    test_df = df[~df['Scan_Name'].str.contains('Retest')]
    retest_df = df[df['Scan_Name'].str.contains('Retest')]
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, feature_cols)

ccc_time = process_time(df_raw[df_raw['exp_type'] == 'R_Time'].copy())


# In Vivo Test-Retest
def process_in_vivo(filepath):
    df = pd.read_csv(filepath).iloc[:, 1:]
    df['TR_type'] = df['Scan_Name'].str[3:]
    test_df = df[df['TR_type'] == 'Test'].copy()
    retest_df = df[df['TR_type'] == 'Retest'].copy()
    test_df['Unique_id'] = test_df['Scan_Name'].str[:6] + '_' + test_df['Segment Name']
    retest_df['Unique_id'] = retest_df['Scan_Name'].str[:6] + '_' + retest_df['Segment Name']
    test_df = test_df.sort_values('Unique_id').reset_index(drop=True)
    retest_df = retest_df.sort_values('Unique_id').reset_index(drop=True)
    return compute_ccc_for_features(test_df, retest_df, df.columns[rad_start:rad_end])

ccc_tr.to_excel('PFCE_intrasession_CCC.xlsx', index=False)
ccc_time.to_excel('PFCE_intersession_CCC.xlsx', index=False)